[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sascha-senko/TensorflowCourse/blob/main/ANNwTFHW7.ipynb)

## Imports

In [1]:
# TODO: Clean up imports
import matplotlib.pyplot as plt
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, \
    BatchNormalization, ELU, Flatten, Dense, ZeroPadding2D, AveragePooling2D, \
    Layer, GlobalAveragePooling2D, concatenate
from tensorflow.keras.regularizers import L2
from functools import partial  
from tensorflow import debugging as debug

UsageError: Line magic function `%tensorflow_version` not found.


## Load dataset

In [ ]:
# TODO: load appropriate dataset. See what needs to be loaded first, though
# Test set is necessary - take 1000 examples. Also, labels of test set, but not of training set, are necessary
x = tfds.load(
    'fashion_mnist', split=['train[:1%]', 'test[:1%]'])

# TODO: remove
print("bla")

## Inspect data set

In [ ]:
fig, ax = plt.subplots(1,10)
shapes = []


# TODO: remove this
num_to_name = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse",
"ship", "truck"]

for i, (img, lbl) in enumerate(train_data):
    if i == 10:
        break
    ax[i].imshow(img, cmap='gray')
    # TODO: remove this
    ax[i].set_title(num_to_name[lbl.numpy()])
    ax[i].axis("off")

## Define some constants

In [ ]:
# arbitrarily set. Feel free to change these
SHUFFLE_SIZE = train_data.cardinality()
PREFETCH_SIZE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128

## Prepare data

In [ ]:
# TODO: only process input
def process_inp_and_label(input, label, training):
    """ rescale inputs and onehotify labels """
    new_input = input / 255
    return new_input, tf.one_hot(label, 10)

# TODO: @Hermann: See what stuff I need to look up online / externally first, note them down and look them up
# See how I could load fashion mnist
# Do I need to process training and test set, or just one?
# TODO: do I actually have to process training and test dataset, or just one?
# TODO: if just one: Remove the argument
# TODO: Follow https://github.com/Maxoz99/ANNs-TensorFlow/blob/main/Homework05/homework05.ipynb for data pipeline
def data_pipeline(data, training):
    """ helper function for data pipeline - does all the things we need """
    # TODO: if only train, remove this partial part and just pass process_in_and_label
    map_func = partial(process_inp_and_label, training=training)
    data = data.map(map_func)
    # arbitrarily set buffer_size - feel free to use sth. else
    data = data.shuffle(buffer_size=SHUFFLE_SIZE)
    data = data.batch(BATCH_SIZE)
    data = data.prefetch(PREFETCH_SIZE)
    return data

# TODO: if only train, remove True argument
train_data = data_pipeline(train_data, True)
# TODO: possibly needs to be changed
test_data = data_pipeline(test_data, False)

## Define Model

### Encoder

In [ ]:
class Encoder(Model):
    # TODO: Implement init. If necessary, add additional arguments to init. 
    # Please use an output dimension of 10 for the final dense layer.
    def __init__():
        super(Encoder, self).init()
        raise NotImplementedError("TODO: Implement Encoder")

    # TODO: Implement call. If necessary, add additional arguments to call
    def call(x):
        raise NotImplementedError("TODO: Define call")     

#### Check Encoder

In [ ]:
test_inp = tf.random.uniform((BATCH_SIZE, 28, 28, 1), maxval=1)
# TODO: @Sascha: Maybe you need to adjust the init arguments?
encoder = Encoder()
out = encoder(x)
# Assert that shape of out is BATCH_SIZE, 10
debug.assert_equal(out.shape, tf.zeros(BATCH_SIZE, 10).shape)

### Decoder

In [ ]:
class Encoder(Model):
    # TODO: Implement init. If necessary, add additional arguments to init. 
    def __init__():
        super(Encoder, self).init()
        raise NotImplementedError("TODO: Implement Encoder")

    # TODO: Implement call. If necessary, add additional arguments to call
    def call(x):
        raise NotImplementedError("TODO: Define call")     

#### Check Decoder

In [ ]:
test_inp = tf.random.uniform((BATCH_SIZE, 10))
# TODO: @Sarah: Maybe you need to adjust the init arguments?
decoder = Decoder()
out = decoder(x)
# Assert that shape of out is BATCH_SIZE, 28, 28, 1
debug.assert_equal(out.shape, tf.zeros(BATCH_SIZE, 28, 28, 1).shape)

In [ ]:
# Leave following things to Hermann: Training, plotting 